Parallelized AHP map matching for KCMMN

In [1]:
# load dependencies'
import concurrent.futures
import pandas as pd
import geopandas as gpd
from shapely.geometry import shape
import osmnx as ox
import networkx as nx
import numpy as np
import requests
import json
from tqdm import tqdm
import matplotlib.pyplot as plt
from urllib.parse import urljoin
from shapely.geometry import Point, LineString, Polygon
import pyproj 
import mm_utils
from datetime import datetime
from IMM import IMM
from Junction_Condition import JC
from MMJ import MMJ
from AHPMM import AHPMM

# import sys
# sys.path.append('..')

In [2]:
import sys
sys.path.append('..')
import preprocessing.downloader as dl

# # load all the data 
# trajectory_number = '1'
# dataset_dir = '../Data/map-matching-dataset/'
# speed_direction_approixmation_dir = '../kcmmn/approximation'
# trajectory_data, nodes_data, edges_data = dl.KCMMN_input_for_fuzzy_AHP('1', dataset_dir=dataset_dir,speed_direction_approximation_dir=speed_direction_approixmation_dir)

In [149]:
%%time
import dask
import dask.bag as db

# outlier_detection_results.append(dask.delayed(sigle_outlier_detection_experiment)(slope, id, perturbated_gdf))

def load_single_trajectory(trajectory_number, dataset_dir,speed_direction_approximation_dir):
    return dl.KCMMN_input_for_fuzzy_AHP(trajectory_number, dataset_dir=dataset_dir,speed_direction_approximation_dir=speed_direction_approixmation_dir)

dataset_dir = '../Data/map-matching-dataset/'
speed_direction_approixmation_dir = '../kcmmn/approximation'

testing_range = range(0, 100)
loading_dataset_bag = []
for i in tqdm(testing_range):
    print(i)
    trajectory_number = str(i)
    loading_dataset_bag.append(dask.delayed(load_single_trajectory)(trajectory_number, dataset_dir=dataset_dir,speed_direction_approximation_dir=speed_direction_approixmation_dir))

100%|██████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 1002.70it/s]

53
CPU times: total: 0 ns
Wall time: 2.97 ms


In [150]:
%%time

from dask.diagnostics import ProgressBar

with ProgressBar():
    loading_dataset_results = dask.compute(*loading_dataset_bag, scheduler='processes')

[########################################] | 100% Completed | 31.11 s
CPU times: total: 672 ms
Wall time: 31.3 s


In [144]:
%%time

def single_AHP_run(trajectory_data, nodes_data, edges_data):
    return AHPMM(trajectory_data, nodes_data, edges_data)

AHP_bag = []
    
for i in tqdm(range(len(testing_range))):
    trajectory_data, nodes_data, edges_data = loading_dataset_results[i]
    
    for j in range(len(trajectory_data)):
        if (trajectory_data['GPS Bearing'][j] <= (np.pi / 2)) and (trajectory_data['GPS Bearing'][j] >= 0 ):
            trajectory_data['GPS Bearing'][j] = (np.pi / 2 - trajectory_data['GPS Bearing'][j]) * 360 / (2 * np.pi)
        else:
            trajectory_data['GPS Bearing'][j] = (5 * np.pi / 2 - trajectory_data['GPS Bearing'][j]) * 360 / (2 * np.pi)    
    
    AHP_bag.append(dask.delayed(single_AHP_run)(trajectory_data, nodes_data, edges_data))

  0%|                                                                                            | 0/1 [00:00<?, ?it/s]<timed exec>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<timed exec>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 10.02it/s]

CPU times: total: 31.2 ms
Wall time: 103 ms


In [145]:
%%time

with ProgressBar():
    AHP_results = dask.compute(*AHP_bag, scheduler='processes')

[########################################] | 100% Completed | 93.21 s
CPU times: total: 188 ms
Wall time: 1min 33s


In [146]:
%%time
matched_links_results = []
matched_links_no_duplicate_results = []
for i in tqdm(range(len(testing_range))):
    matched_links, matched_links_no_duplicate = AHP_results[i]
    matched_links_results.append(matched_links)
    matched_links_no_duplicate_results.append(matched_links_no_duplicate)

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 29.50it/s]

CPU times: total: 31.2 ms
Wall time: 36.9 ms


In [52]:
import os
import dask.bag as db
import algorithms.metric_mm as metric_mm
import preprocessing.constants as constants

ground_truth_dir = dataset_dir
# gt = dl.load_kcmmn_ground_truth(trajectory_number, ground_truth_dir=ground_truth_dir)
# error = mm_utils.evaluate(matched_edge_df,
#     gt,
#     matchid = "geometry")
# print('Error about not uniqufied matched_edge_df: ', error)
errors2 = []
gts = []
for i in tqdm(testing_range):
    # i = 4
    trajectory_number = str(i)
    gt = dl.load_kcmmn_ground_truth(trajectory_number, dataset_dir=ground_truth_dir)
    matched_edge_df = pd.concat(matched_links_no_duplicate_results[i])
    error = mm_utils.evaluate(matched_edge_df,
        gt,
        matchid = "geometry")
    errors2.append(error)
    gts.append(gt)

print('Error about uniqufied matched_edge_df: ', np.mean(errors2))

100%|██████████████████████████████████████████████████████████████████████████████████| 99/99 [00:42<00:00,  2.33it/s]

Error about uniqufied matched_edge_df:  0.19879269071149458


In [56]:
errors2

[0.044922284873669865,
 0.5399509045038684,
 0.2537153987445326,
 0.1651667782709795,
 0.0045827137502002235,
 0.040847285233732676,
 0.0712354730825219,
 0.8847767144647539,
 0.1860455189136754,
 0.462296418124942,
 0.012302921058306963,
 0.06344699971932141,
 0.04692985908389388,
 0.07783050716501477,
 0.8583785870370524,
 0.5960793175500625,
 0.08811151625490568,
 0.10512756640359931,
 0.17923967675419866,
 0.0778673841635232,
 0.03262553737365151,
 0.24853555137620192,
 0.07778643059291364,
 0.8273984510742942,
 0.18658464894641674,
 0.2570771261955198,
 0.022590577335796166,
 0.61680199132967,
 0.1860058893920872,
 0.21298801714553792,
 0.05541489377702164,
 0.3126851324829339,
 0.7661464462645438,
 0.1183405181115485,
 0.4387682185249868,
 0.060010380691745815,
 0.0010427972510829663,
 0.04638352208419701,
 0.03551183645243626,
 0.028283987454093596,
 0.058384425578407305,
 0.044619952678011686,
 0.7915858619487921,
 0.06845620480942528,
 0.0408949632095114,
 0.10922538743178417,

In [59]:
fig = plt.figure(1)
plt.boxplot(errors2,vert=False)
plt.title('AHP (uniqufied) Error %s on ' 
          + str(20) 
          + ' test cases \n(Average error = ' 
          + str(np.average(errors2)) + ')')
plt.show()

In [147]:
ground_truth_dir = dataset_dir
gt = dl.load_kcmmn_ground_truth('60', dataset_dir=ground_truth_dir)
gts=[]
gts.append(gt)

In [148]:
# visualization
%matplotlib tk

idx = 0

# This is how we  visualize edges and error bound 

# plotting edges and starting point together 
f, ax = plt.subplots()

# location for all point
#locs_utm.plot(ax=ax)
point_locs = trajectory_data['geometry'].to_frame()
point_locs.plot(ax = ax, color="Red", markersize = 10)


# this plot all the road system 
edges_data.plot(ax=ax,color = "#bfe4ff", linewidth = 1)

# ground truth
gts[0].plot(ax = ax, linewidth=4, color = 'Yellow')

# result
pd.concat(matched_links_results[0]).plot(ax = ax, color = "Black")

# matched_points_df.plot(ax = ax, color = "Red")

<Axes: >